In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


Firstly, it is necessary to define function which are used in the code later. See their description below.

In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,np.pi)
    phi = np.random.uniform(0,2*np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(phi) * np.cos(theta) 
    Py = mag * np.sin(phi) * np.sin(theta)
    Pz = mag * np.cos(phi)
    
    return Px, Py, Pz
    

def wArr(num):
    '''just for making numpy array of stationary w given the number of events'''
    momenta = np.zeros([num,4])
    momenta[:,3] = wMass
    
    return momenta

def vecToArr(vec):
    '''converts lorentzvector to array of components'''
    arr = np.zeros([4])
    arr[0] = vec.Px()
    arr[1] = vec.Py()
    arr[2] = vec.Pz()
    arr[3] = vec.E()
    
    return arr

def boostedW():
    '''produces numpy array of 4 momenta of w decay into electron and neutrino under a normally 
    distributed boost in z direction'''
    # defining constants
    event = np.zeros([3,4])
    wMass = 80.3
    eE = wMass / 2
    
    # making particles in w rest frame
    thetae, phie = thetaPhi()
    Px, Py, Pz = momenta(eE, thetae, phie)
    w = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(0,0,0, wMass)
    e = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, eE)
    n = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, eE)
    
    # applying boost to all particles
    beta = np.random.normal(0,0.1) # beta (v/c) is normally distributed centred on 0 with width 0.1
    boostedW = r.Math.VectorUtil.boostZ(w, beta)
    wArr = vecToArr(boostedW)
    
    boostedE = r.Math.VectorUtil.boostZ(e, beta)
    eArr = vecToArr(boostedE)
    
    boostedN = r.Math.VectorUtil.boostZ(n, beta)
    nArr = vecToArr(boostedN)
    
    event[0] = wArr
    event[1] = eArr
    event[2] = nArr
    
    return event, beta

Now we can call the function `boostedW()` number of times given by the variable `num`. Events are generated into an array `events` and then each beta by which the particles are boosted is stored into the array `beta`.

In [3]:
# making num events worth of components
num = 1000
events = np.zeros([num, 3, 4])
beta = np.zeros([num])

for i in range(num):
    events[i], beta[i] = boostedW()

Now the TTree is created. This is done in very similar way to the case of W boson at rest. 

One additional branch called `beta` is added to accommodate the values of changing beta. See the `boostValue`.

In [4]:
tree = r.TTree("tree", "initial random e and n components")

wBranch = np.zeros(4)
eBranch = np.zeros(4)
boostValue = np.zeros(4)

tree.Branch("wBoson",wBranch,"wArray[4]/D")
tree.Branch("electron",eBranch,"elArray[4]/D")
tree.Branch("beta", boostValue,"boostArray[4]/D")

Filling the values into their respective branches.

In [5]:
for i in range(num):
    wBranch[2] = events[i,0,2]
    wBranch[3] = events[i,0,3]
    
    for j in range(len(events[0,1])):
        eBranch[j] = events[i,1,j]
    
    boostValue[0] = beta[i]
    tree.Fill()

We can scan the tree to see Values of W boson and electron.

In [6]:
#tree.Scan()

Writing and saving the file.

In [7]:
file = r.TFile("generatedComponentsBOOST.root","recreate")
tree.Write()
file.Close